In [1]:
import json
import math

import numpy as np
import pandas as pd
from scapy.all import rdpcap
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Dataset creation
N = 100 # Num of training samples
L = 1000 # Length of past window
T = 20

x = np.empty((N, L), np.float32)
x[:] = np.array(range(L)) + np.random.randint(-4*T, 4*T, N).reshape(N, 1)
y = np.sin(x/1.0/T).astype(np.float32)



In [3]:
# Defining the model
class LSTMPredictor(nn.Module):
    def __init__(self, n_hidden=51):
        super(LSTMPredictor, self).__init__()

        self.n_hidden = n_hidden

        # Defining 2 LSTMS stacked on top of each other
        self.lstm1 = nn.LSTMCell(1, self.n_hidden) 
        self.lstm2 = nn.LSTMCell(self.n_hidden, self.n_hidden)
        self.linear = nn.Linear(self.n_hidden, 1)

    def forward(self, input, prediction_window=0):
        outputs = []
        n_samples = input.size(0)

        # Initializing the hidden states and cell states
        h_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        c_t = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        h_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)
        c_t2 = torch.zeros(n_samples, self.n_hidden, dtype=torch.float32)

        # Split input tensor into chunks of size, 1, row-wise
        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)
            
        for i in range(prediction_window):
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output)

        # Concatentating all outputs, history + future, row-wise
        outputs = torch.cat(outputs, dim=1)
        return outputs
            

In [4]:
# Training loop

# Initializing training inputs and targest
# We are simply setting the test as the 1 step ground truth. 
# From the 3rd batch to the 100th batch. From the first element to just before the last element
train_input = torch.from_numpy(y[3:, :-1])
# From the 3rd batch to the 100th batch. From the second element to the last element
train_target = torch.from_numpy(y[3:, 1:])

# From the 3rd batch to the 100th batch. From the first element to just before the last element
test_input = torch.from_numpy(y[:3, :-1])
# From the 3rd batch to the 100th batch. From the second element to the last element
test_target = torch.from_numpy(y[:3, 1:])

# Initialising model, loss and optimizer
model = LSTMPredictor()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)



In [5]:
# Initializing training steps
trg_steps = 10

for i in range (trg_steps):
    print("Step: ", i)

    def closure():
        optimizer.zero_grad()
        out = model(train_input)
        loss = criterion(out, train_target)
        print("loss: ", loss.item())
        loss.backward()
        return loss
    
    optimizer.step(closure)

    with torch.no_grad():
        future = 1000
        # Pred includes future values 
        pred = model(test_input, prediction_window=future)
        loss = criterion(pred[:, :-future], test_target)
        print("Test Loss: ", loss.item())
        y = pred.detach().numpy()
        n = train_input.shape[1]

        # print("Pred: ", pred)
        # print("y: ", y[0])
        # print("n: ", n)
        # print("y_i[:n]: ", y[0][:n])
        # print("y_i[n:]: ", y[0][n:])
        
        # print("History: ", y[:-future])
        # print("Future: ", y[future:])

    # # draw the result
    # plt.figure(figsize=(30, 10))
    # plt.title(
    #     'Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
    # plt.xlabel('x', fontsize=20)
    # plt.ylabel('y', fontsize=20)
    # plt.xticks(fontsize=20)
    # plt.yticks(fontsize=20)

    # def draw(yi, color):
    #     plt.plot(np.arange(train_input.size(1)),
    #                 yi[:train_input.size(1)], color, linewidth=2.0)
    #     plt.plot(np.arange(train_input.size(1), train_input.size(1) + future),
    #                 yi[train_input.size(1):], color + ':', linewidth=2.0)
    # draw(y[0], 'r')
    # draw(y[1], 'g')
    # draw(y[2], 'b')
    # plt.savefig('predict%d.pdf' % i)
    # plt.close()

    plt.figure(figsize=(12,6))
    plt.title(f"Step {i+1}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    n = train_input.shape[1] # 999
    print("n: ", n)
    def draw(y_i, color):
        plt.plot(np.arange(n), y_i[:n], color, linewidth=2.0)
        plt.plot(np.arange(n, n+future), y_i[n:], color + ":", linewidth=2.0)
        # print("y_i[:n]: ", y_i[:n])
        # print("y_i[n:]: ", y_i[n:])


    draw(y[0], 'r')
    draw(y[1], 'g')
    draw(y[2], 'b')

    plt.savefig("predict%d.pdf"%i)
    plt.close()

Step:  0
loss:  0.5228500366210938
loss:  0.5132181644439697
loss:  0.4854382872581482
loss:  0.4666913151741028
loss:  0.38999059796333313
loss:  1.3184086084365845
loss:  0.22002696990966797
loss:  0.12816883623600006
loss:  0.07541168481111526
loss:  0.03319218382239342
loss:  0.03266485407948494
loss:  0.03157762065529823
loss:  0.03027687966823578
loss:  0.02739923819899559
loss:  0.022314928472042084
loss:  0.013450122438371181
loss:  0.008019685745239258
loss:  0.007019301410764456
loss:  0.004727658815681934
loss:  0.002284238813444972
Test Loss:  0.0013238618848845363
n:  999
Step:  1
loss:  0.0013153024483472109
loss:  0.0011773802107200027
loss:  0.0010983558604493737
loss:  0.0010599689558148384
loss:  0.000989444786682725
loss:  0.0007088877027854323
loss:  0.0005417756037786603
loss:  0.0005243596970103681
loss:  0.000507023767568171
loss:  0.00047413029824383557
loss:  0.00042707796092145145
loss:  0.00034459971357136965
loss:  0.0002317671023774892
loss:  0.000192775492